Хочу просто показать дополнительные возможности при объявлении аттрибутов объекта:

In [ ]:
function createPerson(first_name, last_name){
    return {
        first_name,
        last_name: last_name,
        ["12" + "3"]: 123 // Как видите я могу объявлять ключ в объекте внутри квадратных скобочек, в них я могу писать выражения
    }
}
var john = createPerson("John", "Dow")
var Andy = createPerson("Andy", "Warhol")
var rand_key = Math.random() > .5 ? "first_name" : "last_name";
john[rand_key]; // Получение случайного ключа из объекта

В объектах мы не только выражаем с вами какие-то структуры, но и используем их как хранилище пар ключ/значение:

> Должен заметить что у нас теперь есть специализированный класс для этого (`Map`)

In [ ]:
var str = "Very very long string...";

var statistic = {}
for (let char of str){
    // если нет ключика char в объекте statistic то добавим его
    if (!(char in statistic)){
        statistic[char] = 0
    }
    statistic[char]++
}
console.log(statistic);

Давайте напиишм функцию которая будет создавать объектики кошки:

In [ ]:
function createCat(nickname){
    return {
        nickname: nickname,
        age: Math.round(12 * Math.random),
        sayHello: function(){
            console.log(`Hello! My name is ${this.nickname}`)
        }
    }
}
var barsik = createCat("Barsik")
var luparik = createCat("Luparik")
barsik.sayHello()
luparik.sayHello()
console.log("Is the same object:", Object.is(barsik.sayHello, luparik.sayHello))

Думаю, что вы видите некоторую странность, у каждого объекта сейчас будет собственная функция `sayHello`, но есть ли более удобный способ делать подобные проекты?

In [ ]:
var additional_prototype = {
    sayName: function(){
        console.log(this.nickname)
    }
}

var cat_prototype = {
    sayMeow: function(){
        console.log(`Meow! My name is ${this.nickname}`)
    },
    __proto__: additional_prototype
}

function createCat(nickname){
    return {
        nickname: nickname,
        age: Math.round(12 * Math.random),
       __proto__: cat_prototype
    }
}

var barsik = createCat("Barsik")
var luparik = createCat("Luparik")
barsik.sayMeow()
luparik.sayMeow()
luparik.sayName()

console.log("Is the same object:", Object.is(barsik.sayHello, luparik.sayHello))

Теперь у нас общая функция на все экземпляры кошек. Мы даже можем поменять функцию в прототипе на новую и поведение метода у всех кошек изменится:

In [ ]:
cat_prototype.sayHello = function(){
    console.log(`[NEW] Hello! My name is ${this.nickname}`)
}

barsik.sayHello()
luparik.sayHello()

Вуаля!

Для того чтобы удобнее создать объект из прототипа, а точнее объект у которого аттрибут `__proto__` будет ссылать на какой-то другой объект можно воспользоваться функцией `Object.create`.

Эта функция получает 1 аргумент:
- `prototype`
     Объект который будет прототипом нового объекта
и возвращает новый объект у которого свойство `__proto__` ссылается на значение аргумента `prototype`

In [ ]:
var person_prototype = {
  isHuman: false,
  printIntroduction: function() {
    console.log(`My name is ${this.name}. Am I human? ${this.isHuman}`);
  }
};

var me = Object.create(person_prototype);
me.name = 'Matthew'; // "name" is a property set on "me", but not on "person"
me.isHuman = true; // inherited properties can be overwritten

me.printIntroduction();

# Функции конструкторы

**Функция конструктор** - функция которая конструирует новые объекты.
> Принято называть с большой буквы

Функция конструктор вызывается через оператор `new`. если вызвана она верно, то контекст выполнения `this` будет равен новому пустому объекту. Функция должна заполнить этот объект аттрибутами.

Оператор `new`:

    - создает пустой объект (`new_obj`)
    - вызывает функцию-конструктор и прокидывает созданный объект как контекст вызова(`this`)
    - связывает `new_obj.__proto__` с аттрибутом `prototype` конструктора

## Example #1 (Cat)

In [ ]:
function Cat(nickname, age){
    this.nickname = nickname
    this.age = age
}
Cat.prototype.sayHello = function(){
    console.log(`Hello! My name is ${this.nickname}`)
}

var luparik = new Cat("Luparik", 12)
var barsik = new Cat("Barsik", 1)
console.log(luparik, barsik)

## Example #2 (Rectangle)

In [ ]:
function Rectangle(width, height){
    this.width = width
    this.height = height
}

Rectangle.prototype.area = function() {
    return this.width * this.height
}

Rectangle.prototype.perimeter = function() {
    return (this.width + this.height) * 2
}

const r1 = new Rectangle(4, 4)
const r2 = new Rectangle(10, 10)

console.log(r1.area()) // 16
console.log(r1.perimeter()) // 16

console.log(r2.area()) // 9
console.log(r2.perimeter()) // 12

## Самостоятельная реализация `new`

In [ ]:
/**
/* Функция повторяет действия оператора `new`
/* @param {Function} class - функция конструктор / класс
/* @return новый объект являющийся экземпляром конструктора
*/
function _new(cls, args){
    const o = Object.create(cls.prototype)
    cls.apply(o, args)
    return o
}

const r3 = _new(Rectangle, [5, 5])
r3.width === 5 // true
r3.heigth === 5 // true
r3.__proto__ === Rectangle.prototype // true

## Очень очень плохой пример кода!
👇 Внизу вы увидете настоящую вакханалию, но она призвана показать, что функции-конструкторы являются самыми объчными функциями, которые можно вызывать и без `new`. Так что вот вам функция `Adder` которая при обычном вызове просто суммирует значение аргументов, а при вызове через `new` конструирует объект в контексте вызова `this`:

In [ ]:
function Adder(a, b){
    if (this === global){
        return a + b
    }
    this.a = a
    this.b = b
}

Adder.prototype.add = function () {
    return this.a + this.b
}

console.log(Adder(5, 10))
let a = new Adder(5, 20)
console.log(a.__proto__ === Adder.prototype)


# Наследование классов

В этом примере мы реализуем цепочку прототипов:

In [4]:
function A(){
    this.is_a = true
}

A.prototype.a = function(){
    console.log("Method from A")
}

{
    const a = new A()
    a.a() // "Method from A"
    a.is_a // true
}


Method from A


true

In [6]:
function B(){
    this.is_b = true
}

B.prototype.b = function(){
    console.log("Method from B")
}

B.prototype.__proto__ = A.prototype
// Object.setPrototypeOf(B.prototype, A.prototype)

{
    const b = new B()
    b.b()
    b.a()
    cosnole.log("b.is_a", b.is_a)
}

Method from B
Method from A


In [ ]:
{
    function C(){
        this.is_c = true
    }

    C.prototype.c = function(){
        console.log("Method from C")
    }

    Object.setPrototypeOf(C.prototype, B.prototype)
}

In [2]:
{
    const c = new C()
    c.a()
    c.b()
    c.c()
}

Method from A
Method from A
Method from B
Method from C


In [2]:
class Persone{
    constructor(first_name, last_name){
        this.first_name = first_name
        this.last_name = last_name
    }

    get fullname(){
        return `${this.first_name} ${this.last_name}`
    }

    set fullname(value){
        [this.first_name, this.last_name] = value.split(" ")
    }
}

{
    const p = new Persone("John", "Dow")
    console.log(p.fullname)
    p.fullname = "Andy Warhol"
    console.log(p.first_name)
    console.log(p.last_name)
}

John Dow
Andy
Warhol


Сейчас в нашем примере есть некоторая проблема, заключающаяся в том, что у объекта класса `C` отсутствуют свойства `is_a` и `is_b` потому что, функции-конструкторы `A` и `B` просто не вызывались.

Прежде чем попытаться реализовать правильную работу классов, нам следует разобраться с несколькими методами:
- `Function.prototype.bind()`
- `Function.prototype.apply()`
- `Function.prototype.call()`

# Явное прокидывание контекста `this`
Перед нами функция `show_this` - которая выводит в консоль значение контекста и значения 2 аргументов: `a` и `b`:

In [3]:
function show_this(a, b){
    console.log("this:", this)
    console.log("argument 'a':", a)
    console.log("argument 'b':", b)
}

При обычном вызове этой функции контест вызова будет ссылаться на глобальный объект (_в нашем случае на объект модуля `global`_)

In [ ]:
show_this(4, 6)

Для того чтобы вызвать функцию с явным указанием контеста у функции есть 2 метода:
- `Function.prototype.apply`
- `Funciton.prototype.call`

Давайте посмотрим на следующие примеры:

In [11]:
show_this.apply({a: 1}, [1, 2])

this: [String: 'sdfsdf']
argument 'a': 1
argument 'b': 2


In [ ]:
const my_super_obj = {cool_attr: Number.MAX_VALUE}
show_this.call(my_super_obj, 1, 2)

У нас остался еще один, близкий к двум предыдущим метод. `Function.prototype.bind` - получает значение контекста и возвращает новую функцию у которой контекст уже будет привязан к переданному объекту.

In [16]:
{
    let b_show_this = show_this.bind({}, 1, 2)
    b_show_this()
}

this: {}
argument 'a': 1
argument 'b': 2


In [19]:
show_this.bind({}, 1, 2)()

this: {}
argument 'a': 1
argument 'b': 2


In [23]:
{
    const nf = show_this.bind({})
    nf(1,2)
}

this: {}
argument 'a': 1
argument 'b': 2


In [24]:
show_this.bind("Super string", 3, 4)(5, 6)

this: [String: 'Super string']
argument 'a': 3
argument 'b': 4


In [9]:
{
    function bind(func, thisArg, ...args){
        return function(..._args){
            return func.apply(thisArg, [...args, ..._args])
        }
    }

    new Array() instanceof Object

    bind(show_this, "Super string", 3, 4)()
    bind(show_this, "Super string")(5, 6)
}

true
this: {}
argument 'a': undefined
argument 'b': undefined
true
this: [String: 'Super string']
argument 'a': 3
argument 'b': 4
true
this: [String: 'Super string']
argument 'a': 5
argument 'b': 6


In [5]:
const adder = (a) => (b) => a + b

console.log(adder(1)(2))

3


## Уже верная реализация наследования классов

In [ ]:
function A(){
    this.is_a = true
}

A.prototype.a = function(){
    console.log("Method from A")
}

// === B ===

function B(...args){
    A.apply(this, args)
    this.is_b = true
}

B.prototype.b = function(){
    console.log("Method from B")
}

Object.setPrototypeOf(B.prototype, A.prototype)

// === C ===

function C(){Ï
    B.apply(this, args)
    this.is_c = true
}

C.prototype.c = function(){
    console.log("Method from C")
}

Object.setPrototypeOf(C.prototype, B.prototype)

c = new C()
c.a()
c.b()
c.c()

In [ ]:
class A {
    constructor() {
        this.is_a = true
    }

    a(){
        console.log("Method a from 'A'")
    }
}

class B extends A {
    constructor() {
        super()
        this.is_b = true
    }
    
    b(){
        console.log("Method b from 'B'")
    }
}


class C extends B {
    constructor() {
        super()
        this.is_c = true
    }
    
    c(){
        console.log("Method c from 'C'")
    }
}

In [17]:
{
    class Geometry {
        geometries = []
        constructor(){
            this.is_geometry = true
            Geometry.geometries.push(this)
        }
        area(){
            throw new Error("Method area is not implemented")
        }
        perim(){
            throw new Error(`Method [${this.__proto__.constructor.name}].perim is not implemented`)
        }
    }

    class Reactangle extends Geometry {
        constructor(width, heigth){
            super()
            this.width = width
            this.heigth = heigth
        }
        area(){
            return this.width * this.heigth
        }
        perim(){
            return (this.width + this.heigth) * 2
        }
    }

    class Circle extends Geometry {
        constructor (d) {
            super()
            this.d = d
        }
        
        // 2πr
        // p(r^2)
    }
}

600
[
  Reactangle { is_geometry: true, width: 20, heigth: 30 },
  Reactangle { is_geometry: true, width: 40, heigth: 50 }
]
